In [ ]:
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine('sqlite:///programming.sqlite')

In [ ]:
# What are the tables in the database?

# Save the table names to a list: table_names
table_names  = engine.table_names()

# Print the table names to the shell
print(table_names)

# See the data model here: http://chinookdatabase.codeplex.com/wikipage?title=Chinook_Schema&referringTitle=Home

In [ ]:
# The power of SQL lies in relationships between tables: INNER JOIN

# Execute query and store records in DataFrame: df
itp_rep = pd.read_sql_query("select * from Grade WHERE Disciplina = 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO' AND Situaçăo like '%REPROVADO%' ",engine)
itp_ap = pd.read_sql_query("select * from Grade WHERE Disciplina = 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO' AND Situaçăo like '%APROVADO%' ",engine)
itp_tra = pd.read_sql_query("select * from Grade WHERE Disciplina = 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO' AND Situaçăo like '%CANCELADO%' ",engine)
dfAll = pd.read_sql_query("select * from Grade",engine) 

# drop the unnecessary column
itp_rep = itp_rep.drop('index', 1)
itp_ap = itp_ap.drop('index', 1)
dfAll = dfAll.drop('index', 1)

itp_rep = itp_rep[['Aluno_ID', 'Período']].drop_duplicates().groupby('Período').count()
itp_ap = itp_ap[['Aluno_ID', 'Período']].drop_duplicates().groupby('Período').count()

# Print head of DataFrame

dfAll

In [ ]:
# Import modules from Bokeh visualization library
from bokeh.charts import Scatter, BoxPlot, Bar, color, Donut
from bokeh.layouts import gridplot, row
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource


p = Bar(itp_rep, values='Aluno_ID', xlabel='Período', ylabel='Total', 
        title="Students failing in ITP by period", color='Aluno_ID')
output_notebook()
show(p)

In [ ]:
p2 = Bar(itp_ap, values='Aluno_ID', xlabel='Período', ylabel='Total', 
        title="Students Approved in ITP by period", color='Aluno_ID')

output_notebook()
show(p2)

## Subjects Connections

In [ ]:
# Execute query and store records in DataFrame: df
base_df = pd.read_sql_query('SELECT * FROM Grade', engine)

# drop the unnecessary column
base_df = base_df.drop('index', 1)

In [ ]:
base_df.loc[base_df['Disciplina'] == 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO', 'Disciplina'] = 'ITP'
base_df.loc[base_df['Disciplina'] == 'IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO', 'Disciplina']   = 'PTP'
base_df.loc[base_df['Disciplina'] == 'IMD0029 - ESTRUTURA DE DADOS BÁSICAS I', 'Disciplina']            = 'EDB1'
base_df.loc[base_df['Disciplina'] == 'IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I', 'Disciplina']              = 'LP1'
base_df.loc[base_df['Disciplina'] == 'IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II', 'Disciplina']          = 'EDB2'
base_df.loc[base_df['Disciplina'] == 'IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II', 'Disciplina']             = 'LP2'

In [ ]:
# Based on main subjects set its dependents
SECONDARY_SUBJECTS = ['EDB1', 'EDB2', 'LP1', 'LP2']

In [ ]:
# Import modules from Bokeh visualization library
from bokeh.charts import Scatter, BoxPlot, Bar, color, Donut
from bokeh.layouts import gridplot, row
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource

In [ ]:
# Add the lib package to the system path, so that we can include SubjectConnections from there.

import os
import sys
module_path = os.path.abspath(os.path.join('./'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from lib.subject_connections import SubjectConnections

In [ ]:
def get_group_by_grade(grade):
    if(grade >= 5 and grade < 6):
        return '5 < 6'
    if(grade >= 6 and grade < 7):
        return '6 < 7'
    if(grade >= 7 and grade < 8):
        return '7 < 8'
    if(grade >= 8 and grade < 9):
        return '8 < 9'
    if(grade >= 9):
        return '9 - 10'

In [ ]:
# Generate bar chart by students grouped df

def students_grouped_bar(df, title):
    return Bar(
            df,
            label  = 'Grupo Média',
            values = 'Total',
            title  = title,
            xlabel = 'Grade groups',
            ylabel = 'Total approved students',
            color  = 'Grupo Média',
            legend='top_right',
            plot_height=300
            
    )

In [ ]:
# Helper method to generate boxplot

def generate_boxplot(df, base_subject_key, subject_key): 
    return BoxPlot(
        df,
        values = '{}_grade'.format(subject_key),
        label  = 'Grupo Média',
        color  = 'Grupo Média',
        xlabel = '{} grade groups'.format(base_subject_key),
        ylabel = '{} grades'.format(subject_key), 
        title  = '{} student grades grouped by {} grades'.format(subject_key, base_subject_key)
    )

In [ ]:
# Generate boxplot list based on SubjectConnections instance

def dependent_subjects_list(list, subj_connection, base_subject):
    boxplot_list = []

    for subject in list:
        column_name = '{}_grade'.format(subject)

        students_df = base_df[base_df['Disciplina'] == subject]
        df = subj_connection.obtain_values_from(students_df, column_name, ['Grupo Média'])
        
        grade_groups_mean_df = df.groupby('Grupo Média')[column_name].mean().reset_index()
        
        box = generate_boxplot(df, base_subject, subject)
        box.line(x = grade_groups_mean_df['Grupo Média'], y = grade_groups_mean_df[column_name], line_width = 2, line_color = '#2C3E50')
        box.circle(x = grade_groups_mean_df['Grupo Média'], y = grade_groups_mean_df[column_name], color = "#F1C40F", size = 5)

        boxplot_list.append(box)
    
    return boxplot_list

### ITP - INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO

In [ ]:
query = """
    SELECT *
    FROM Grade
    WHERE Disciplina = 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO'
    AND Situaçăo LIKE 'APROVADO%'
"""

ITP_students_df = pd.read_sql_query(query, engine)

ITP_connections = SubjectConnections(ITP_students_df, 'Aluno_ID')
ITP_connections.parse_column('Média Final', 'Grupo Média', get_group_by_grade)

In [ ]:
total_approved_ITP_by_group = ITP_connections.base_df.groupby('Grupo Média', as_index=False)['Aluno_ID'].agg({
    'Total': 'count'
})

b = students_grouped_bar(total_approved_ITP_by_group, 'Total approved students on ITP by grade group')

output_notebook()
show(b)

In [ ]:
boxplots = dependent_subjects_list(
    SECONDARY_SUBJECTS, ITP_connections, 'ITP'
)

grid = gridplot(boxplots, ncols=2, plot_width=460, plot_height=400)

output_notebook()
show(grid)

### PTP - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO

In [ ]:
query = """
    SELECT *
    FROM Grade
    WHERE Disciplina = 'IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO'
    AND Situaçăo LIKE 'APROVADO%'
"""

PTP_students_df = pd.read_sql_query(query, engine)

PTP_connections = SubjectConnections(PTP_students_df, 'Aluno_ID')
PTP_connections.parse_column('Média Final', 'Grupo Média', get_group_by_grade)

In [ ]:
total_approved_PTP_by_group = PTP_connections.base_df.groupby('Grupo Média', as_index=False)['Aluno_ID'].agg({
    'Total': 'count'
})

b = students_grouped_bar(total_approved_PTP_by_group, 'Total approved students on PTP by grade group')

output_notebook()
show(b)

In [ ]:
boxplots = dependent_subjects_list(
    SECONDARY_SUBJECTS, PTP_connections, 'PTP'
)

grid = gridplot(boxplots, ncols=2, plot_width=460, plot_height=400)

output_notebook()
show(grid)

## Students Reproved on Subjects

In [ ]:
def stack_dataframe_column(df, column_label, column_value):
    stacked = df.reset_index().drop('index', 1).stack()
    final_df = pd.DataFrame(stacked).reset_index(1)
    final_df.rename(columns={ 'level_1': column_label, 0: column_value }, inplace=True)
    
    return final_df

def stack_dataframe_by_units(df, units = []):
    frames = []

    for unit in units:
        frames.append(stack_dataframe_column(df[unit], 'Unidade', 'Nota'))
    
    return pd.concat(frames)

In [ ]:
def generate_reproved_analysis_on_subject(subject, subject_alias):
    query = """
        SELECT *
        FROM Grade
        WHERE Disciplina = '{}'
        AND Situaçăo LIKE 'REPROVADO%'
    """
    query = query.format(subject)

    reproved_students_df = pd.read_sql_query(query, engine)
    units_df = stack_dataframe_by_units(reproved_students_df, ['Unidade I', 'Unidade II ', 'Unidade III'])
    
    return BoxPlot(
        units_df,
        values='Nota',
        label='Unidade',
        color='Unidade',
        title='Units grades of students reproved in {}'.format(subject_alias)
    )

In [ ]:
boxplots = []

subjects = [
    { 'title': 'IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO', 'alias': 'ITP' },
    { 'title': 'IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO',   'alias': 'PTP' },
    { 'title': 'IMD0029 - ESTRUTURA DE DADOS BÁSICAS I',            'alias': 'EDB 1' },
    { 'title': 'IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II',          'alias': 'EDB 2' },
    { 'title': 'IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I',              'alias': 'LP1' },
    { 'title': 'IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II',             'alias': 'LP2' }
]

for subject in subjects:
    boxplots.append(generate_reproved_analysis_on_subject(subject['title'], subject['alias']))

In [ ]:
grid = gridplot(boxplots, ncols=2, plot_width=460, plot_height=400)

output_notebook()
show(grid)